In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from PIL import Image
import math
import cv2
import os
import tensorflow as tf
from sklearn.utils import class_weight
import random
from sklearn.preprocessing import OneHotEncoder
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.models import Sequential
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications import InceptionV3
from sklearn.metrics import confusion_matrix, classification_report
from tensorflow.keras.models import Model
from keras.applications.vgg16 import VGG16
from keras.callbacks import ReduceLROnPlateau, ModelCheckpoint
import math

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import albumentations as alb

In [4]:
mura_df1 = pd.read_csv('/content/drive/MyDrive/Anamoly_Det/Anamoly_Det/Mura_final.csv')

In [5]:
print("Sample of the MURA dataset:")
print(mura_df1.head())

Sample of the MURA dataset:
                                               Names  Labels
0  MURA-v1.1/train/XR_SHOULDER/patient00001/study...       1
1  MURA-v1.1/train/XR_SHOULDER/patient00001/study...       1
2  MURA-v1.1/train/XR_SHOULDER/patient00001/study...       1
3  MURA-v1.1/train/XR_SHOULDER/patient00002/study...       1
4  MURA-v1.1/train/XR_SHOULDER/patient00002/study...       1


In [6]:
mura_df =[]
for i in range (len(mura_df1)):
    if 'SHOULDER' in mura_df1.iloc[i,0]:
        mura_df.append(mura_df1.iloc[i,:])
mura_df= pd.DataFrame(mura_df)

In [7]:
mura_df.reset_index(inplace=True)

In [ ]:
mura_df

,index,Names,Labels
0,0,MURA-v1.1/train/XR_SHOULDER/patient00001/study...,1
1,1,MURA-v1.1/train/XR_SHOULDER/patient00001/study...,1
2,2,MURA-v1.1/train/XR_SHOULDER/patient00001/study...,1
3,3,MURA-v1.1/train/XR_SHOULDER/patient00002/study...,1
4,4,MURA-v1.1/train/XR_SHOULDER/patient00002/study...,1
...,...,...,...
8937,39074,MURA-v1.1/valid/XR_SHOULDER/patient11272/study...,0
8938,39075,MURA-v1.1/valid/XR_SHOULDER/patient11272/study...,0
8939,39076,MURA-v1.1/valid/XR_SHOULDER/patient11801/study...,0
8940,39077,MURA-v1.1/valid/XR_SHOULDER/patient11801/study...,0


In [ ]:
# Checking for missing values
print("Missing values in the MURA dataset:")
print(mura_df.isnull().sum())

Missing values in the MURA dataset:
index     0
Names     0
Labels    0
dtype: int64


In [8]:
mura_df['Labels'].value_counts()

Labels
0    4496
1    4446
Name: count, dtype: int64

In [ ]:
# img = lambda im_name : np.asarray(Image.open(im_name).convert("RGB"))

In [ ]:
# def preprocessing_img(im):
#     transform = alb.Resize(224,224)
#     im = transform(image=im)['image']
#     im = np.array(im)
#     im = im/255
#     return im

In [9]:
x = []
counter = 0
for i in mura_df['Names']:
    # if counter >= 10000:  # Check if we've loaded 1000 images
    #     break
    if 'train' in i:
        i = i.replace('MURA-v1.1/train/', '/content/drive/MyDrive/Anamoly_Det/Anamoly_Det/')
    elif 'valid' in i:
        i = i.replace('MURA-v1.1/valid/', '/content/drive/MyDrive/Anamoly_Det/Anamoly_Det/')
    else:
        pass
    # print(i)
    x.append(i)
    # try:
    #   x.append(preprocessing_img(img(i)))
    # except NameError:
    #   print("Variable x is not defined")
    # except:
    #   print("Something else went wrong")


    # counter += 1  # Increment counter

# print("Total images loaded:", counter)

In [10]:
x=pd.Series(x)

In [11]:
print(x)

0       /content/drive/MyDrive/Anamoly_Det/Anamoly_Det...
1       /content/drive/MyDrive/Anamoly_Det/Anamoly_Det...
2       /content/drive/MyDrive/Anamoly_Det/Anamoly_Det...
3       /content/drive/MyDrive/Anamoly_Det/Anamoly_Det...
4       /content/drive/MyDrive/Anamoly_Det/Anamoly_Det...
                              ...                        
8937    /content/drive/MyDrive/Anamoly_Det/Anamoly_Det...
8938    /content/drive/MyDrive/Anamoly_Det/Anamoly_Det...
8939    /content/drive/MyDrive/Anamoly_Det/Anamoly_Det...
8940    /content/drive/MyDrive/Anamoly_Det/Anamoly_Det...
8941    /content/drive/MyDrive/Anamoly_Det/Anamoly_Det...
Length: 8942, dtype: object


In [12]:
y=mura_df['Labels']

In [ ]:
y

0       1
1       1
2       1
3       1
4       1
       ..
8937    0
8938    0
8939    0
8940    0
8941    0
Name: Labels, Length: 8942, dtype: int64

In [ ]:
x

0       /content/drive/MyDrive/Anamoly_Det/Anamoly_Det...
1       /content/drive/MyDrive/Anamoly_Det/Anamoly_Det...
2       /content/drive/MyDrive/Anamoly_Det/Anamoly_Det...
3       /content/drive/MyDrive/Anamoly_Det/Anamoly_Det...
4       /content/drive/MyDrive/Anamoly_Det/Anamoly_Det...
                              ...                        
8937    /content/drive/MyDrive/Anamoly_Det/Anamoly_Det...
8938    /content/drive/MyDrive/Anamoly_Det/Anamoly_Det...
8939    /content/drive/MyDrive/Anamoly_Det/Anamoly_Det...
8940    /content/drive/MyDrive/Anamoly_Det/Anamoly_Det...
8941    /content/drive/MyDrive/Anamoly_Det/Anamoly_Det...
Length: 8942, dtype: object

In [13]:
df=pd.concat([x,y],axis=1)

In [14]:
df = df.sample(frac = 1)

In [ ]:
df

,0,Labels
7845,/content/drive/MyDrive/Anamoly_Det/Anamoly_Det...,0
1384,/content/drive/MyDrive/Anamoly_Det/Anamoly_Det...,1
4146,/content/drive/MyDrive/Anamoly_Det/Anamoly_Det...,1
6180,/content/drive/MyDrive/Anamoly_Det/Anamoly_Det...,0
1252,/content/drive/MyDrive/Anamoly_Det/Anamoly_Det...,1
...,...,...
8245,/content/drive/MyDrive/Anamoly_Det/Anamoly_Det...,0
7799,/content/drive/MyDrive/Anamoly_Det/Anamoly_Det...,0
5972,/content/drive/MyDrive/Anamoly_Det/Anamoly_Det...,0
243,/content/drive/MyDrive/Anamoly_Det/Anamoly_Det...,1


In [15]:
df['Labels'] = df['Labels'].replace({0: 'negative', 1: 'positive'})

In [ ]:
#df = df.head(6900)

In [16]:
train_df, val_df = train_test_split(df,random_state=42, shuffle = True, train_size=0.8)

In [17]:
train_df['Labels'].value_counts()

Labels
negative    3608
positive    3545
Name: count, dtype: int64

In [18]:
train_data_gen = ImageDataGenerator(
    rescale = 1./255,
    rotation_range=20,
    width_shift_range = 0.1,
    height_shift_range = 0.1,
    shear_range = 0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

In [19]:
train_generater = train_data_gen.flow_from_dataframe(
    dataframe = train_df,
    x_col = 0,
    y_col = 'Labels',
    batch_size=54,
    class_mode ='binary',
    color_mode='rgb',
    target_size = (224,224),
    shuffle=True

)

Found 7150 validated image filenames belonging to 2 classes.


/usr/local/lib/python3.10/dist-packages/keras/src/preprocessing/image.py:1137: UserWarning: Found 3 invalid image filename(s) in x_col="0". These filename(s) will be ignored.
  warnings.warn(


In [20]:
# Get the number of batches
num_batches = len(train_generater)

# Initialize empty lists to store images and labels
images = []
labels = []

# Loop through all batches to extract images and labels
for i in range(num_batches):
    batch = train_generater.next()
    images.extend(batch[0])
    labels.extend(batch[1])

# Convert lists to NumPy arrays
images_array = np.array(images)
labels_array = np.array(labels)

In [ ]:
np.save('train_img.npy',images_array)

In [ ]:
np.save('train_img_labels.npy',labels_array)

In [ ]:
for i in images_array:
  print(i.shape)

VGG 16 MODEL

In [ ]:
from keras.applications import VGG16
from keras.layers import Input
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.models import Model

# Load the pre-trained VGG16 model without the top (fully connected layers)
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224,224,3), pooling='avg')

# Freeze the weights of the pre-trained layers
# for layer in base_model.layers:
#     layer.trainable = False

# model = Sequential()

# model.add(base_model)

# model.add(Flatten())

# model.add(Dense(4096, activation='relu'))
# model.add(Dropout(0.5))
# model.add(Dense(4096, activation='relu'))
# model.add(Dropout(0.5))

# model.add(Dense(1, activation='sigmoid'))
flat = Flatten()(base_model.output)
out_layer = Dense(1, activation='sigmoid')(flat)
model = Model(inputs=base_model.input, outputs=out_layer)
# Compile the model
model.compile(optimizer=Adam(lr=0.0001), loss='binary_crossentropy', metrics=['accuracy'])

# Print model summary
model.summary()

# Create a new input layer for grayscale images
# grayscale_input = Input(shape=(None, None, 1))

# Create a new model with the modified input layer
# Iterate through the layers of the VGG16 model and transfer them to the new model
# x = Conv2D(3, (3, 3), activation='relu', padding='same')(grayscale_input)
# for layer in vgg16_base.layers[1:]:
#     x = layer(x)

# Add new fully connected layers
# x = Flatten()(x)
# x = Dense(4096, activation='relu')(x)
# x = Dense(4096, activation='relu')(x)
# output = Dense(1000, activation='softmax')(x)

# Create the new model
# vgg16_grayscale = Model(inputs=grayscale_input, outputs=output)

# Freeze the weights of the pre-trained layers
# for layer in vgg16_grayscale.layers[:2]:
#     layer.trainable = False

# Compile the model
# vgg16_grayscale.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Summary of the model
# vgg16_grayscale.summary()


58889256/58889256 [==============================] - 0s 0us/step


Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0   

In [ ]:
from keras.callbacks import ModelCheckpoint

# Define checkpoint callback
checkpoint = ModelCheckpoint('best_model.h5', monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')

# Assuming you have your training and validation data ready, train the model
history = model.fit(images_array,
                    labels_array,
                    epochs=20,
                    validation_split=0.1,
                    batch_size=54)

# After training, you can load the best model using:
# model.load_weights('best_model.h5')


Epoch 1/20
92/92 [==============================] - 131s 1s/step - loss: 1.2335 - accuracy: 0.5044 - val_loss: 0.6931 - val_accuracy: 0.5109
Epoch 2/20
92/92 [==============================] - 64s 696ms/step - loss: 0.6932 - accuracy: 0.4988 - val_loss: 0.6932 - val_accuracy: 0.4891
Epoch 3/20
92/92 [==============================] - 64s 700ms/step - loss: 0.6931 - accuracy: 0.5068 - val_loss: 0.6933 - val_accuracy: 0.4891
Epoch 4/20
92/92 [==============================] - 64s 699ms/step - loss: 0.6931 - accuracy: 0.5068 - val_loss: 0.6933 - val_accuracy: 0.4891
Epoch 5/20
92/92 [==============================] - 64s 699ms/step - loss: 0.6931 - accuracy: 0.5068 - val_loss: 0.6934 - val_accuracy: 0.4891
Epoch 6/20
92/92 [==============================] - 64s 698ms/step - loss: 0.6931 - accuracy: 0.5068 - val_loss: 0.6934 - val_accuracy: 0.4891
Epoch 7/20
92/92 [==============================] - 64s 698ms/step - loss: 0.6931 - accuracy: 0.5068 - val_loss: 0.6934 - val_accuracy: 0.4891
E

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
# Assuming you have trained the model already, now predict the labels for validation data
y_val_pred = model.predict(X_test)

# Convert predicted probabilities to class labels
y_val_pred_classes=(y_val_pred > np.mean(y_val_pred)).astype(int)

# Compute accuracy
accuracy = accuracy_score(y_test, y_val_pred_classes)

# Compute precision
precision = precision_score(y_test, y_val_pred_classes, average='weighted')  # Set zero_division to 1

# Compute recall
recall = recall_score(y_test, y_val_pred_classes, average='weighted')

# Compute F1 score
f1 = f1_score(y_test, y_val_pred_classes, average='weighted')

# Compute confusion matrix
cm = confusion_matrix(y_test, y_val_pred_classes)

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)
print("Confusion Matrix:",cm)
print (np.mean(y_val_pred))

44/44 [==============================] - 13s 155ms/step
Accuracy: 0.5036231884057971
Precision: 0.253636315900021
Recall: 0.5036231884057971
F1 Score: 0.33736685873930505
Confusion Matrix: [[695   0]
 [685   0]]
0.49271744


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


DESNSENET 169 MODEL

In [21]:
from keras.applications import DenseNet169
from keras.layers import Dense, GlobalAveragePooling2D
from keras.models import Model
from keras.optimizers import Adam

# Load the pre-trained DenseNet169 model without the top (fully connected layers)
base_model = DenseNet169(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Add a global average pooling layer
x = base_model.output
x = GlobalAveragePooling2D()(x)

# Add a fully connected layer
x = Dense(1024, activation='relu')(x)

# Add the output layer
predictions = Dense(1, activation='sigmoid')(x)

# Create the model
model = Model(inputs=base_model.input, outputs=predictions)

# Freeze the weights of the pre-trained layers
for layer in base_model.layers:
    layer.trainable = False

# Compile the model
model.compile(optimizer=Adam(lr=0.0001), loss='binary_crossentropy', metrics=['accuracy'])

# Print model summary
model.summary()


51877672/51877672 [==============================] - 3s 0us/step


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 224, 224, 3)]        0         []                            
                                                                                                  
 zero_padding2d (ZeroPaddin  (None, 230, 230, 3)          0         ['input_1[0][0]']             
 g2D)                                                                                             
                                                                                                  
 conv1/conv (Conv2D)         (None, 112, 112, 64)         9408      ['zero_padding2d[0][0]']      
                                                                                                  
 conv1/bn (BatchNormalizati  (None, 112, 112, 64)         256       ['conv1/conv[0][0]']      

In [22]:
# Define checkpoint callback
checkpoint = ModelCheckpoint('best_model_densenet.h5', monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')

# Assuming you have your training and validation data ready, train the model
history = model.fit(images_array,
                    labels_array,
                    epochs=20,
                    validation_split=0.1,
                    batch_size=54,
                    callbacks=[checkpoint])

# After training, you can load the best model using:
# model.load_weights('best_model_densenet.h5')

Epoch 1/20
120/120 [==============================] - ETA: 0s - loss: 0.6936 - accuracy: 0.6186
Epoch 1: val_accuracy improved from -inf to 0.58881, saving model to best_model_densenet.h5


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


120/120 [==============================] - 52s 291ms/step - loss: 0.6936 - accuracy: 0.6186 - val_loss: 0.7330 - val_accuracy: 0.5888
Epoch 2/20
119/120 [============================>.] - ETA: 0s - loss: 0.5967 - accuracy: 0.6853
Epoch 2: val_accuracy improved from 0.58881 to 0.68112, saving model to best_model_densenet.h5
120/120 [==============================] - 25s 210ms/step - loss: 0.5968 - accuracy: 0.6850 - val_loss: 0.5995 - val_accuracy: 0.6811
Epoch 3/20
119/120 [============================>.] - ETA: 0s - loss: 0.5702 - accuracy: 0.7053
Epoch 3: val_accuracy improved from 0.68112 to 0.72448, saving model to best_model_densenet.h5
120/120 [==============================] - 26s 220ms/step - loss: 0.5702 - accuracy: 0.7054 - val_loss: 0.5561 - val_accuracy: 0.7245
Epoch 4/20
119/120 [============================>.] - ETA: 0s - loss: 0.5488 - accuracy: 0.7162
Epoch 4: val_accuracy improved from 0.72448 to 0.72727, saving model to best_model_densenet.h5
120/120 [================

In [23]:
val_generater = train_data_gen.flow_from_dataframe(
    dataframe = val_df,
    x_col = 0,
    y_col = 'Labels',
    batch_size=54,
    class_mode ='binary',
    color_mode='rgb',
    target_size = (224,224),
    shuffle=True

)

Found 1789 validated image filenames belonging to 2 classes.


In [24]:
# Get the number of batches
num_batches = len(val_generater)

# Initialize empty lists to store images and labels
images = []
labels = []

# Loop through all batches to extract images and labels
for i in range(num_batches):
    batch = val_generater.next()
    images.extend(batch[0])
    labels.extend(batch[1])

# Convert lists to NumPy arrays
X_test = np.array(images)
y_test = np.array(labels)

In [25]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
# Assuming you have trained the model already, now predict the labels for validation data
y_val_pred = model.predict(X_test)

# Convert predicted probabilities to class labels
y_val_pred_classes=(y_val_pred > np.mean(y_val_pred)).astype(int)

# Compute accuracy
accuracy = accuracy_score(y_test, y_val_pred_classes)

# Compute precision
precision = precision_score(y_test, y_val_pred_classes, average='weighted')  # Set zero_division to 1

# Compute recall
recall = recall_score(y_test, y_val_pred_classes, average='weighted')

# Compute F1 score
f1 = f1_score(y_test, y_val_pred_classes, average='weighted')

# Compute confusion matrix
cm = confusion_matrix(y_test, y_val_pred_classes)

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)
print("Confusion Matrix:",cm)
print (np.mean(y_val_pred))

56/56 [==============================] - 20s 213ms/step
Accuracy: 0.703745108999441
Precision: 0.7046813007582245
Recall: 0.703745108999441
F1 Score: 0.7035220846388535
Confusion Matrix: [[651 237]
 [293 608]]
0.4808877


RESNET 50 MODEL

In [ ]:
from keras.applications import ResNet50
from keras.layers import Dense, GlobalAveragePooling2D
from keras.models import Model
from keras.optimizers import Adam

# Load the pre-trained ResNet50 model without the top (fully connected layers)
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Add a global average pooling layer
x = base_model.output
x = GlobalAveragePooling2D()(x)

# Add a fully connected layer
x = Dense(1024, activation='relu')(x)

# Add the output layer
predictions = Dense(1, activation='sigmoid')(x)

# Create the model
model = Model(inputs=base_model.input, outputs=predictions)

# Freeze the weights of the pre-trained layers
for layer in base_model.layers:
    layer.trainable = False

# Compile the model
model.compile(optimizer=Adam(lr=0.0001), loss='binary_crossentropy', metrics=['accuracy'])

# Print model summary
model.summary()


94765736/94765736 [==============================] - 3s 0us/step


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 224, 224, 3)]        0         []                            
                                                                                                  
 conv1_pad (ZeroPadding2D)   (None, 230, 230, 3)          0         ['input_1[0][0]']             
                                                                                                  
 conv1_conv (Conv2D)         (None, 112, 112, 64)         9472      ['conv1_pad[0][0]']           
                                                                                                  
 conv1_bn (BatchNormalizati  (None, 112, 112, 64)         256       ['conv1_conv[0][0]']          
 on)                                                                                          

In [ ]:
# Define checkpoint callback
checkpoint = ModelCheckpoint('best_model_resnet.h5', monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')

# Assuming you have your training and validation data ready, train the model
history = model.fit(images_array,
                    labels_array,
                    epochs=20,
                    validation_split=0.1,
                    batch_size=44,
                    callbacks=[checkpoint])


Epoch 1/20
113/113 [==============================] - ETA: 0s - loss: 0.7581 - accuracy: 0.4984
Epoch 1: val_accuracy improved from -inf to 0.53442, saving model to best_model_resnet.h5


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


113/113 [==============================] - 30s 188ms/step - loss: 0.7581 - accuracy: 0.4984 - val_loss: 0.7008 - val_accuracy: 0.5344
Epoch 2/20
113/113 [==============================] - ETA: 0s - loss: 0.7071 - accuracy: 0.5058
Epoch 2: val_accuracy improved from 0.53442 to 0.54529, saving model to best_model_resnet.h5
113/113 [==============================] - 16s 140ms/step - loss: 0.7071 - accuracy: 0.5058 - val_loss: 0.6888 - val_accuracy: 0.5453
Epoch 3/20
113/113 [==============================] - ETA: 0s - loss: 0.6983 - accuracy: 0.5199
Epoch 3: val_accuracy did not improve from 0.54529
113/113 [==============================] - 16s 137ms/step - loss: 0.6983 - accuracy: 0.5199 - val_loss: 0.7215 - val_accuracy: 0.5362
Epoch 4/20
113/113 [==============================] - ETA: 0s - loss: 0.7043 - accuracy: 0.5260
Epoch 4: val_accuracy did not improve from 0.54529
113/113 [==============================] - 16s 141ms/step - loss: 0.7043 - accuracy: 0.5260 - val_loss: 0.7108 - va

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
# Assuming you have trained the model already, now predict the labels for validation data
y_val_pred = model.predict(X_test)

# Convert predicted probabilities to class labels
y_val_pred_classes=(y_val_pred > np.mean(y_val_pred)).astype(int)

# Compute accuracy
accuracy = accuracy_score(y_test, y_val_pred_classes)

# Compute precision
precision = precision_score(y_test, y_val_pred_classes, average='weighted')  # Set zero_division to 1

# Compute recall
recall = recall_score(y_test, y_val_pred_classes, average='weighted')

# Compute F1 score
f1 = f1_score(y_test, y_val_pred_classes, average='weighted')

# Compute confusion matrix
cm = confusion_matrix(y_test, y_val_pred_classes)

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)
print("Confusion Matrix:",cm)
print (np.mean(y_val_pred))

44/44 [==============================] - 7s 103ms/step
Accuracy: 0.5768115942028985
Precision: 0.5896207608776466
Recall: 0.5768115942028985
F1 Score: 0.5675702184319998
Confusion Matrix: [[306 403]
 [181 490]]
0.47460195
